<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [3]:
# Импортируем всякое нужное
import pandas as pd
import psycopg2
import warnings
import requests
from bs4 import BeautifulSoup
# Избавимся от лишних warning
warnings.filterwarnings('ignore')

In [4]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [5]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies).

In [6]:
# Посчитаем число строк в vacancies = кол-ву вакансий
query_3_1 = f'''
SELECT COUNT(*)
FROM vacancies
'''

In [7]:
# результат запроса
jobs = pd.read_sql_query(query_3_1, connection)
print(f'Всего выбрано {jobs.iloc[0,0]} вакансий')

Всего выбрано 49197 вакансий


2. Напишите запрос, который посчитает количество работодателей (таблица employers).

In [10]:
# Посчитаем число строк в employers = кол-ву работодателей
query_3_2 = f'''
SELECT COUNT(*)
FROM employers
'''

In [11]:
# результат запроса
emps = pd.read_sql_query(query_3_2, connection)
print(f'Всего выбрано {emps.iloc[0,0]} работодателей')

Всего выбрано 23501 работодателей


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [12]:
# Посчитаем число строк в areas = кол-ву регионов
query_3_3 = f'''
SELECT COUNT(*)
FROM areas
'''

In [13]:
# результат запроса
regions = pd.read_sql_query(query_3_3, connection)
print(f'Всего выбрано {regions.iloc[0,0]} регионов')

Всего выбрано 1362 регионов


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [14]:
# Посчитаем число строк в industries = кол-ву сфер деятельности
query_3_4 = f'''
SELECT COUNT(*)
FROM industries
'''

In [15]:
# результат запроса
inds = pd.read_sql_query(query_3_4, connection)
print(f'Всего выбрано {inds.iloc[0,0]} сфер деятельности')

Всего выбрано 294 сфер деятельности


***

In [16]:
# выводы по предварительному анализу данных
print(
    '''
    - Всего рассматриваемых вакансий - {0}.
    - Количество зарегистрированных работодателей составляет {1}.
    - Выбрано {2} различных регионов.
    - В рассматриваемой выборке {3} различных сфер деятельности.

        Из первых двух пунктов можно предположить, что в среднем один работодатель
    предлагает по две вакансии. Также можно ответить большое разнообразие
    представленных регионов и сфер деятельности - эти данные помогут провести
    полноценный анализ.
    '''.format(jobs.iloc[0, 0], emps.iloc[0, 0],
               regions.iloc[0, 0], inds.iloc[0, 0])
)


    - Всего рассматриваемых вакансий - 49197.
    - Количество зарегистрированных работодателей составляет 23501.
    - Выбрано 1362 различных регионов.
    - В рассматриваемой выборке 294 различных сфер деятельности.

        Из первых двух пунктов можно предположить, что в среднем один работодатель
    предлагает по две вакансии. Также можно ответить большое разнообразие
    представленных регионов и сфер деятельности - эти данные помогут провести
    полноценный анализ.
    


# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [17]:
# Выберем количество вакансий в каждом регионе, соединим таблицы и отсортируем
query_4_1 = f'''
SELECT COUNT(*) cnt, a.name area
FROM vacancies v
JOIN areas a ON a.id = v.area_id
GROUP BY area
ORDER BY cnt DESC
'''

In [18]:
# результат запроса
count_vacs = pd.read_sql_query(query_4_1, connection)
display(count_vacs)

,cnt,area
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы
...,...,...
764,1,Тарко-Сале
765,1,Новоаннинский
766,1,Бирск
767,1,Сасово


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [19]:
# Выведем кол-во вакансий, где указана зарплата (любая)
query_4_2 = f'''
SELECT COUNT(*)
FROM vacancies
WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL
'''

In [20]:
# результат запроса
vacs_with_salaries = pd.read_sql_query(query_4_2, connection)
print('''
      В списке вакансий, где указано хотя бы одно поле с зарплатой - {0} вакансии.
      '''.format(vacs_with_salaries.iloc[0,0])
      )


      В списке вакансий, где указано хотя бы одно поле с зарплатой - 24073 вакансии.
      


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [21]:
# Выведем средние значения минимальной и максимальной зарплат
query_4_3 = f'''
SELECT
    v.salary_from,
    v.salary_to
FROM vacancies v
WHERE
    /*фильтр salary_from не пустое или salary_to не пустое значение*/
    v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL
    '''

In [22]:
# результат запроса
avg_salaries = pd.read_sql_query(query_4_3, connection)
salary_from = round(avg_salaries['salary_from'].mean())
salary_to = round(avg_salaries['salary_to'].mean())

print(f'Cредние значения для нижней границы зарплатной вилки {salary_from}.')
print(f'Cредние значения для верхней границы зарплатной вилки {salary_to}.')


Cредние значения для нижней границы зарплатной вилки 71065.
Cредние значения для верхней границы зарплатной вилки 110537.


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [23]:
# Выведем количество вакансий для различных сочетаний графика и типа трудоустройства
query_4_4 = f'''
SELECT COUNT(*) vacancies, schedule, employment
FROM vacancies
GROUP BY schedule, employment
ORDER BY COUNT(*) DESC
'''

In [24]:
schedule_employment = pd.read_sql_query(query_4_4, connection)
display(schedule_employment)

,vacancies,schedule,employment
0,35367,Полный день,Полная занятость
1,7802,Удаленная работа,Полная занятость
2,1593,Гибкий график,Полная занятость
3,1312,Удаленная работа,Частичная занятость
4,940,Сменный график,Полная занятость
5,569,Полный день,Стажировка
6,367,Вахтовый метод,Полная занятость
7,347,Полный день,Частичная занятость
8,312,Гибкий график,Частичная занятость
9,141,Полный день,Проектная работа


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.

In [25]:
# Выведем количество вакансий, соответствующих тому или иному опыту работы
query_4_5 = f'''
SELECT experience, COUNT(*) vacancies
FROM vacancies
GROUP BY experience
ORDER BY vacancies ASC
'''

In [26]:
experience = pd.read_sql_query(query_4_5, connection)
display(experience)

,experience,vacancies
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

In [27]:
# выводы по детальному анализу вакансий
print('''
    1) Было определено количество вакансий по городам. В пятёрку лидеров входят:
    {1}: {0}, {3}: {2}, {5}: {4}, {7}: {6},
    {9}: {8}.

    2) У {10} вакансий указан уровень зарплаты, что составляет около {11}% из общего
    числа.

    3) Средний уровень зарплаты по всем вакансиям находится в диапазоне
    от {12} рублей до {13} рублей.

    4) Было выведено количество вакансий по признакам "Рабочий график" и "Тип занятости".
    Первое место - {15} и {16}: {14} вакансий.
    На второй позиции - {18} и {19}: {17} вакансий.
    Можно предположить, что востребованость в удалённой работе растёт и в будущем
    этот показатель может только вырасти.

    5) Наиболее востребованный опыт работы у работодателей {20}.
    Можно сделать вывод: работодатели заинтересованы в сотрудниках с опытом работы,
    т.к. потратят меньше ресурсов на нового работника, также такие сотрудники более самостоятельны
    в принятии решений и исполнении должностных обязанностей.
'''.format(count_vacs.iloc[0][0], count_vacs.iloc[0][1], count_vacs.iloc[1][0], count_vacs.iloc[1][1],
           count_vacs.iloc[2][0], count_vacs.iloc[2][1], count_vacs.iloc[3][0], count_vacs.iloc[3][1],
           count_vacs.iloc[4][0], count_vacs.iloc[4][1], vacs_with_salaries.iloc[0,0],
           round(vacs_with_salaries.iloc[0,0]/jobs.iloc[0][0]*100), salary_from, salary_to,
           schedule_employment.iloc[0][0], schedule_employment.iloc[0][1], schedule_employment.iloc[0][2], schedule_employment.iloc[1][0],
           schedule_employment.iloc[1][1], schedule_employment.iloc[1][2], experience.iloc[3][0])
      )


    1) Было определено количество вакансий по городам. В пятёрку лидеров входят:
    Москва: 5333, Санкт-Петербург: 2851, Минск: 2112, Новосибирск: 2006,
    Алматы: 1892.

    2) У 24073 вакансий указан уровень зарплаты, что составляет около 49% из общего
    числа.

    3) Средний уровень зарплаты по всем вакансиям находится в диапазоне
    от 71065 рублей до 110537 рублей.

    4) Было выведено количество вакансий по признакам "Рабочий график" и "Тип занятости".
    Первое место - Полный день и Полная занятость: 35367 вакансий.
    На второй позиции - Удаленная работа и Полная занятость: 7802 вакансий.
    Можно предположить, что востребованость в удалённой работе растёт и в будущем
    этот показатель может только вырасти.

    5) Наиболее востребованный опыт работы у работодателей От 1 года до 3 лет.
    Можно сделать вывод: работодатели заинтересованы в сотрудниках с опытом работы,
    т.к. потратят меньше ресурсов на нового работника, также такие сотрудники более самостоятельны

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [28]:
# Пойдем простым путём - найдем топ-5 работодателей по количеству вакансий через
# соединение таблицы вакансий и таблицы работодателей
query_5_1 = f'''
SELECT
    e.name employer,
    count(v.id) vacancies
FROM employers e
JOIN vacancies v ON v.employer_id = e.id
GROUP BY employer
ORDER BY vacancies DESC
LIMIT 5
'''

In [29]:
# результат запроса
employers_vacs = pd.read_sql_query(query_5_1, connection)
print('''
    Работодатель "{0}" находится на первом месте по количеству вакансий.
    "{1}" - на пятом месте.'''.format(employers_vacs['employer'][0], employers_vacs['employer'][4]))


    Работодатель "Яндекс" находится на первом месте по количеству вакансий.
    "Газпром нефть" - на пятом месте.


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [30]:
# Соберём таблицу с регионами и кол-вом вакансий, а также количеством работодателей в них.
query_5_2 = f'''
SELECT a.name area,
COUNT(DISTINCT e.id) employers,
COUNT(DISTINCT v.id) vacancies
FROM areas a
LEFT JOIN employers e ON e.area = a.id
LEFT JOIN vacancies v ON v.area_id = a.id
GROUP BY a.id
ORDER BY employers
'''
# Среди регионов, где нет вакансий, найдем регион с наибольшим числом работодателей
query_5_2_2 = f'''
SELECT a.name area,
COUNT(DISTINCT e.id) employers,
COUNT(DISTINCT v.id) vacancies
FROM areas a
LEFT JOIN public.employers e ON e.area = a.id
LEFT JOIN public.vacancies v ON v.area_id = a.id
WHERE v.id IS NULL
GROUP BY a.id
ORDER BY employers DESC
LIMIT 1
'''


In [31]:
# результат запроса
area_info = pd.read_sql_query(query_5_2, connection)
display(area_info)
zero_vacancies_regions = pd.read_sql_query(query_5_2_2, connection)
display(zero_vacancies_regions)

,area,employers,vacancies
0,Верхневилюйск,0,0
1,Порхов,0,0
2,Пустошка,0,0
3,Пыталово,0,0
4,Дорогобуж,0,1
...,...,...,...
1357,Екатеринбург,609,1698
1358,Алматы,721,1892
1359,Минск,1115,2112
1360,Санкт-Петербург,2217,2851


,area,employers,vacancies
0,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [32]:
# Посчитаем кол-во регионов, где публикуются вакансии (в разрезе работодателей)
query_5_3 = f'''
SELECT e.name employer,
COUNT(DISTINCT v.area_id) cnt
FROM employers e
JOIN vacancies v ON v.employer_id = e.id
GROUP BY e.name
ORDER BY cnt DESC
'''

In [33]:
# результат запроса
employer_area = pd.read_sql_query(query_5_3, connection)
display(employer_area)

,employer,cnt
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.

In [34]:
# Соединим таблицы, чтобы вывести число работодателей, у которых не указана сфера деятельности
query_5_4 = f'''
SELECT
COUNT(e.id) employers_count
FROM employers e
LEFT JOIN employers_industries i ON e.id = i.employer_id
WHERE i.industry_id IS NULL
'''

In [35]:
# результат запроса
no_industry_employers = pd.read_sql_query(query_5_4, connection)
print('''
    У {0} работодателей в анкете не указана сфера деятельности.'''
      .format(no_industry_employers.iloc[0, 0]))



    У 8419 работодателей в анкете не указана сфера деятельности.


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

In [36]:
# Выберем компании с 4 сферами деятельности и выведем третью после сортировки
query_5_5 = f'''
SELECT e.name employer
FROM employers e
LEFT JOIN employers_industries i ON i.employer_id = e.id
GROUP BY e.name
HAVING COUNT(i.industry_id) = 4
ORDER BY e.name
LIMIT 1 OFFSET 2
'''

In [37]:
# результат запроса
four_industries_employer = pd.read_sql_query(query_5_5, connection)
print('''
    Компания "{0}" находится на третьем месте среди работодателей, у кого указано
    четыре сферы деятельности.'''.format(four_industries_employer.iloc[0, 0]))


    Компания "2ГИС" находится на третьем месте среди работодателей, у кого указано
    четыре сферы деятельности.


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [38]:
# Выберем компании, занимающиеся разработкой программного обеспечения и посчитаем
query_5_6 = f'''
SELECT COUNT(e.id) employers_number
FROM employers_industries i
JOIN employers e ON i.employer_id = e.id
JOIN industries ii ON ii.id = i.industry_id
WHERE ii.name = 'Разработка программного обеспечения'
'''

In [39]:
# результат запроса
specific_industry_employers = pd.read_sql_query(query_5_6, connection)
print('''
    {0} работодателей указали «Разработку программного обеспечения»
    в качестве сферы деятельности.'''.format(specific_industry_employers.iloc[0, 0]))


    3553 работодателей указали «Разработку программного обеспечения»
    в качестве сферы деятельности.


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8).

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API.

In [44]:
# код для получения списка городов-милионников
# Читаем ссылку и вперёд
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
table = pd.read_html(url)[0]
cities_list = list(table['Город'])
display(cities_list)

['Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Красноярск',
 'Нижний Новгород',
 'Челябинск',
 'Уфа',
 'Самара',
 'Ростов-на-Дону',
 'Краснодар',
 'Омск',
 'Воронеж',
 'Пермь',
 'Волгоград']

In [45]:
# Найдем кол-во вакансий 'Яндекс' в городах-миллионниках, потом сгруппируем данные по городам,
# и объединим получившееся с общим числом вакансий
query_5_7 = f'''
SELECT a.name, COUNT(v.id) cnt
FROM vacancies v
JOIN areas a ON a.id = v.area_id
JOIN employers e ON e.id = v.employer_id
WHERE e.name = 'Яндекс' AND a.name IN ('Москва', 'Санкт-Петербург', 'Новосибирск',
'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара',
'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')
GROUP BY a.name
UNION ALL
SELECT 'Total', COUNT(v.id)
FROM vacancies v
JOIN areas a ON a.id = v.area_id
JOIN employers e ON e.id = v.employer_id
WHERE e.name = 'Яндекс' AND a.name IN ('Москва', 'Санкт-Петербург', 'Новосибирск',
'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара',
'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')
ORDER BY cnt ASC
'''

In [46]:
# результат запроса
yandex_areas = pd.read_sql_query(query_5_7, connection)
display(yandex_areas)

,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

In [47]:
# выводы по анализу работодателей
print('''
    По нашим данным заметно, что абсолютный лидер по вакансиям в it-сфере - "{0}".
    Компания предлагает больше всего вакансий в каждом из регионов.
    Также в топе находятся другие широко известные компании: "{1}", "{2}", "{3}",
    "{4}" и так далее. При этом, если говорить о средних предлагаемых зарплатах
    (из указанных), мы видим, что "{0}" на одном из последних мест, а последнее
    место - занимает "Авито". Это может говорить о том, что зачастую крупные
    компании набирают больше всего людей на небольшие должности с небольшим окладом.
    {5} работодателей не указали сферы деятельности. Возможная причина - работа требует
    от соискателя высоких профессиональных навыков в различных областях.
        '''
      .format(employers_vacs.iloc[0,0], employers_vacs.iloc[1,0], employers_vacs.iloc[2,0],
              employers_vacs.iloc[3,0], employers_vacs.iloc[4,0], no_industry_employers.iloc[0, 0]
      ))


    По нашим данным заметно, что абсолютный лидер по вакансиям в it-сфере - "Яндекс".
    Компания предлагает больше всего вакансий в каждом из регионов.
    Также в топе находятся другие широко известные компании: "Ростелеком", "Тинькофф", "СБЕР",
    "Газпром нефть" и так далее. При этом, если говорить о средних предлагаемых зарплатах
    (из указанных), мы видим, что "Яндекс" на одном из последних мест, а последнее
    место - занимает "Авито". Это может говорить о том, что зачастую крупные
    компании набирают больше всего людей на небольшие должности с небольшим окладом.
    8419 работодателей не указали сферы деятельности. Возможная причина - работа требует
    от соискателя высоких профессиональных навыков в различных областях.
        


# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.*


In [48]:
# Искать вакансии будем по ключу, для удобства переведем в нижний регистр
query_6_1 = f'''
SELECT COUNT(v.id) cnt
FROM vacancies v
WHERE lower(v.name) LIKE '%data%' OR lower(v.name) LIKE'%данн%'
'''

In [50]:
# результат запроса
data_vacancies  = pd.read_sql_query(query_6_1,connection)
print('''
    Под указанные критерии подходит {0} вакансия.'''.format(data_vacancies.iloc[0]['cnt']))


    Под указанные критерии подходит 1771 вакансия.


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста?
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.


In [51]:
# Будем искать по ключам, дополнительно добавим ключ 'junior' и значения в 'опыт'
query_6_2 = f'''
SELECT COUNT(v.name) cnt
FROM vacancies v
WHERE (lower(v.name) LIKE '%data scientist%'
OR lower(v.name) LIKE '%data science%'
OR lower(v.name) LIKE '%исследователь данных%'
OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
OR lower(v.name) LIKE '%machine learning%'
OR lower(v.name) LIKE '%машинн%обучен%')
AND ((lower(v.name) LIKE '%junior%') OR (v.experience='Нет опыта') OR (v.employment='Стажировка'))
'''

In [52]:
# результат запроса
beginner_vacancies = pd.read_sql_query(query_6_2, connection)
print('''
    Выбрано {0} подходящих вакансий для начинающего дата-сайентиста.'''.format(beginner_vacancies.iloc[0]['cnt']))


    Выбрано 51 подходящих вакансий для начинающего дата-сайентиста.


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [53]:
# Ищем по тому же принципу, что и в предыдущем задании, но использовать будем key_skills
query_6_3 = f'''
SELECT COUNT(id)
FROM vacancies
WHERE
    ( name ILIKE '%data scientist%' OR
      name ILIKE '%data science%' OR
      name ILIKE '%исследователь данных%' OR
      name ILIKE '%machine learning%' OR
      name ILIKE '%машинн%обучен%' OR
      name ILIKE '%ML%' AND name NOT ILIKE '%HTML%' )
    AND
    ( key_skills ILIKE '%SQL%' OR key_skills ILIKE '%postgres%' )
'''

In [54]:
# результат запроса
sql_vacancies = pd.read_sql_query(query_6_3, connection)
print('''
        В {0} вакансиях для DS в качестве ключа указан навык SQL или postgres.'''.format(sql_vacancies.iloc[0][0]))


        В 229 вакансиях для DS в качестве ключа указан навык SQL или postgres.


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [55]:
# То же самое, но используем другие навыки
query_6_4 = f'''
SELECT COUNT(*)
FROM vacancies v
WHERE
    (
    name ILIKE '%data scientist%'  OR
    name ILIKE  '%data science%' OR
    name ILIKE  '%исследователь данных%'  OR
    (name ILIKE  '%ML%' AND name NOT ILIKE '%HTML%')  OR
    name ILIKE '%machine learning%' OR
    name ILIKE '%машинн%обучен%'
    )
    AND
    key_skills ILIKE '%Python%'
'''

In [56]:
python_vacancies = pd.read_sql_query(query_6_4, connection)
print('''
    В {0} вакансиях для DS в качестве ключевого навыка указан Python.'''.format(python_vacancies.iloc[0][0]))


    В 357 вакансиях для DS в качестве ключевого навыка указан Python.


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [57]:
# Посчитаем среднее. Так как навыки разделены табуляцией - удалим их (заменим пустой строкой)
# После вычтем из полной строки сокращённую и прибавим 1
query_6_5 = f'''
SELECT ROUND(AVG(1+(LENGTH(v.key_skills) - LENGTH(COALESCE(REPLACE(v.key_skills, CHR(9), ''))))), 2)
FROM vacancies v
WHERE (lower(v.name) LIKE '%data scientist%'
    OR lower(v.name) LIKE '%data science%'
    OR lower(v.name) LIKE '%исследователь данных%'
    OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
    OR lower(v.name) LIKE '%machine learning%'
    OR lower(v.name) LIKE '%машинн%обучен%')
    AND v.key_skills IS NOT NULL
'''

In [58]:
key_skills_count = pd.read_sql_query(query_6_5, connection)
print('''
    Для вакансий DS в среднем указывают {0} ключевых навыков.'''.format(key_skills_count.iloc[0][0]))


    Для вакансий DS в среднем указывают 6.41 ключевых навыков.


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*).

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа.

In [59]:
# Для расчета смотрим только те вакансии, в которых указана какая-нибудь зарплата, сгруппируем по опыту
query_6_6 = f'''
SELECT DISTINCT v.experience, ROUND(AVG(COALESCE(((salary_from + salary_to)/2), salary_from, salary_to)))
FROM vacancies v
WHERE (lower(v.name) LIKE '%data scientist%'
    OR lower(v.name) LIKE '%data science%'
    OR lower(v.name) LIKE '%исследователь данных%'
    OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
    OR lower(v.name) LIKE '%machine learning%'
    OR lower(v.name) LIKE '%машинн%обучен%')
    AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
    GROUP BY v.experience
'''

In [60]:
avg_ds_salary = pd.read_sql_query(query_6_6, connection)
display(avg_ds_salary)

,experience,round
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

In [61]:
# выводы по предметному анализу
print('''
По итогам исследования можно сказать, что вакансий для начинающих среди всех вакансий DS достаточно мало ({1} вакансия из {0}).
Можно предположить, что новичкам приходится значительно труднее, нежели работникам с опытом.
По зарплатам видно, что сотрудникам без опыта предлагают вдвое меньшую среднюю зарплату, чем тем, кто имеет опыт от 1 до 3 лет.
Среди ключевых навыков (которых в среднем {4}) достаточно популярен Python - его требуют в {3} вакансиях. SQL/Postgres также популярен,
данный навык указан в {2} вакансиях. Из вышесказанного можно сделать вывод, что среди сотрудников без опыта в этой области предполагается
высокая конкуренция из-за небольшого количества подходящих вакансий. Среднее количество ключевых навыков (в топе - Python и SQL) также показывает,
что от специалистов DS любого уровня ожидаются довольно обширные знания и навыки.
'''.format(data_vacancies.iloc[0][0], beginner_vacancies.iloc[0][0], sql_vacancies.iloc[0][0],
               python_vacancies.iloc[0][0], key_skills_count.iloc[0][0], avg_ds_salary)
)


По итогам исследования можно сказать, что вакансий для начинающих среди всех вакансий DS достаточно мало (51 вакансия из 1771).
Можно предположить, что новичкам приходится значительно труднее, нежели работникам с опытом.
По зарплатам видно, что сотрудникам без опыта предлагают вдвое меньшую среднюю зарплату, чем тем, кто имеет опыт от 1 до 3 лет.
Среди ключевых навыков (которых в среднем 6.41) достаточно популярен Python - его требуют в 357 вакансиях. SQL/Postgres также популярен,
данный навык указан в 229 вакансиях. Из вышесказанного можно сделать вывод, что среди сотрудников без опыта в этой области предполагается
высокая конкуренция из-за небольшого количества подходящих вакансий. Среднее количество ключевых навыков (в топе - Python и SQL) также показывает,
что от специалистов DS любого уровня ожидаются довольно обширные знания и навыки.



# Общий вывод по проекту

In [62]:
# Дополнительно можно найти количество вакансий DS с режимом работы 'на удаленке' (в данном случае можно не привязываться к региону)
query_7_1 = f'''
SELECT COUNT(v.id) cnt
FROM vacancies v
WHERE schedule = 'Удаленная работа'
AND (lower(v.name) LIKE '%data scientist%'
    OR lower(v.name) LIKE '%data science%'
    OR lower(v.name) LIKE '%исследователь данных%'
    OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
    OR lower(v.name) LIKE '%machine learning%'
    OR lower(v.name) LIKE '%машинн%обучен%')
    '''

In [63]:
distant_ds_job = pd.read_sql_query(query_7_1, connection)
print('''
    Удалённая работа указана в {0} вакансиях DS.'''.format(distant_ds_job.iloc[0][0]))


    Удалённая работа указана в 115 вакансиях DS.


In [64]:
# Если с удалённой работой не получилось - посмотрим топ-20 регионов, где предлагают вакансии DS
query_7_2 = f'''
SELECT a.name, COUNT(v.id) cnt
FROM areas a
JOIN vacancies v ON v.area_id = a.id
WHERE (lower(v.name) LIKE '%data scientist%'
    OR lower(v.name) LIKE '%data science%'
    OR lower(v.name) LIKE '%исследователь данных%'
    OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
    OR lower(v.name) LIKE '%machine learning%'
    OR lower(v.name) LIKE '%машинн%обучен%')
GROUP BY a.name
ORDER BY cnt DESC
LIMIT 20
'''

In [66]:
# Выведем регионы с вакансиями DS
ds_area_vacancies = pd.read_sql_query(query_7_2, connection)
display(ds_area_vacancies)

,name,cnt
0,Москва,217
1,Санкт-Петербург,64
2,Новосибирск,23
3,Нижний Новгород,20
4,Алматы,16
5,Казань,16
6,Минск,9
7,Томск,8
8,Екатеринбург,7
9,Нур-Султан,6


Исходя из результатов дополнительных исследований - для DS не последнюю роль играет регион проживания, так как только в 115 из 1771 вакансий указана удалённая работа (однако возможно, что в некоторых случаях удаленная работа возможна после интервью или прохождения испытательного срока).

Лидером по количеству вакансий DS является Москва, далее идут Санкт-Петербург, Новосибирск, Нижний Новогород и другие города России. За рубежом вакансии есть в Казахстане, Беларуси, Армении, Узбекистане и Кипре. У проживающих в указанных регионах (или готовых туда переехать) соискателей более высокие шансы найти подходящую вакансию DS. Но нельзя исключать и удаленную работу, хотя, конкуренция на эти позиции будет выше, с большой долей вероятности.


## **Заключение**
Подытожив все исследования можно сделать общие выводы:

      1. Общий анализ вакансий
      - Больше всего вакансий размещены в крупных городах;
      - Примерно в половине вакансий указан размер оплаты труда;
      - Самые распространенный график работы - "Полный день и полная занятость". Второе место занимает "Удалённая работа и полная занятость";
      - Больше всего вакансий предлагается специалистам с опытом от 1 года до 3 лет.
      
      2. Анализ работодателей
      - Больше всего вакансий размещают крупные компании из России, преимущественно в крупных городах;
      - Больше трети компаний не имеет определённой сферы деятельности;
      - Примерно 15% компаний занимаются "Разработкой программного обспечения";
      
      3. Предметный анализ
      - Среди 1771 вакансий, относящихся к DS, для начинающих специалистов доступна только 51 вакансия;
      - Из наиболее популярных ключевых навыков можно выделить Python и SQL;
      - В среднем специалисту DS нужно обладать 6 ключевыми навыками;
      - Средняя зарплата специалиста DS - около 180тыс. и напрямую зависит от опыта.

Учитывая вышеизложенное можно вывести следующее:
- Не смотря на то, что специальность DS не является самой популярной (исходя из предоставленных данных), вакансий достаточно как для новичков, так и для соискателей с опытом. Наличие хотя бы одного года работы уже значительно увеличивает число вакансий DS.
- Распределение вакансий по регионам довольно прозрачное: лучше всего искать работу в Москве или других крупных городах (также стоит рассмотреть популярные места релокации - например страны СНГ). Вакансии с возможностью удаленной работы составляют всего 6,5% от общего числа. На текущий момент времени сложно предположить, будет ли расти процент вакансий с удаленной работой.
